In [21]:
import numpy as np
import pandas as pd
import math
import sys
from numpy.polynomial import polynomial as P

# ==============================================================================
# CÁC HÀM CÔNG CỤ (TÁI SỬ DỤNG TỪ CODE NEWTON)
# ==============================================================================

def format_polynomial(coeffs, var_name='x'):
    """Tạo chuỗi biểu diễn đa thức từ mảng hệ số [a₀, a₁, a₂, ...]."""
    if np.all(np.abs(coeffs) < 1e-12): return "0.0"
    
    poly_str = []
    # Xử lý hệ số tự do a₀
    if abs(coeffs[0]) > 1e-12:
        poly_str.append(f"{coeffs[0]:.8f}")
        
    # Xử lý các hệ số còn lại a₁, a₂, ...
    for i in range(1, len(coeffs)):
        if abs(coeffs[i]) > 1e-12:
            sign = " + " if coeffs[i] > 0 else " - "
            coeff_val = abs(coeffs[i])
            
            if i == 1:
                poly_str.append(f"{sign}{coeff_val:.8f}*{var_name}")
            else:
                poly_str.append(f"{sign}{coeff_val:.8f}*{var_name}^{i}")
    
    result = "".join(poly_str).strip()
    if result.startswith("+ "):
        result = result[2:]
    return result

def read_data_from_excel(file_path):
    """Đọc dữ liệu từ file Excel, hỗ trợ cả dấu phẩy thập phân."""
    try:
        # Thêm decimal=',' để tương thích với định dạng số của Việt Nam
        df = pd.read_excel(file_path, decimal=',')
        if 'x' not in df.columns or 'y' not in df.columns:
            return None, "LỖI: File phải chứa 2 cột 'x' và 'y'."
        # Chuyển đổi và loại bỏ dòng lỗi
        df['x'] = pd.to_numeric(df['x'], errors='coerce')
        df['y'] = pd.to_numeric(df['y'], errors='coerce')
        df.dropna(subset=['x', 'y'], inplace=True)
        return df, f"✔ Đã đọc thành công {len(df)} dòng dữ liệu."
    except FileNotFoundError:
        return None, f"LỖI: Không tìm thấy file '{file_path}'."
    except Exception as e:
        return None, f"LỖI: Không thể đọc file. Chi tiết: {e}"

def check_if_equally_spaced(x_data, tolerance=1e-9):
    """Kiểm tra mốc cách đều."""
    if len(x_data) < 2: return False, None
    diffs = np.diff(x_data)
    return np.all(np.abs(diffs - diffs[0]) < tolerance), diffs[0]

def run_central_interpolation(x_full, y_full, h):
    while True:
        try:
            num_points = int(input(f"\nNhập số điểm nội suy cần dùng (Bessel/Gauss nên dùng chẵn, Stirling nên dùng lẻ): "));
            if 2 <= num_points <= len(x_full): break
            else: print(f"LỖI: Vui lòng nhập số điểm trong khoảng [2, {len(x_full)}].")
        except ValueError: print("LỖI: Vui lòng nhập một số nguyên.")
            
    while True:
        try:
            x_target = float(input("Nhập giá trị x cần tính toán (x_bar): ")); break
        except ValueError: print("LỖI: Vui lòng nhập một số thực.")

    while True:
        method_choice = input("\nChọn thuật toán nội suy trung tâm:\n1. Gauss I\n2. Gauss II\n3. Stirling\n4. Bessel\nLựa chọn (1-4): ")
        if method_choice in ['1', '2', '3', '4']: break

    method_names = {'1': 'Gauss I', '2': 'Gauss II', '3': 'Stirling', '4': 'Bessel'}
    print(f"\n-> Bắt đầu tính toán với phương pháp {method_names[method_choice]}")

    print("\n\n=======================================================================")
    print("B1: XÁC ĐỊNH MỐC GỐC x₀ VÀ TRÍCH XUẤT MỐC (THEO THUẬT TOÁN)")
    print("=======================================================================")
    
    # --- LOGIC CHỌN MỐC GỐC ĐÚNG THEO TỪNG PHƯƠNG PHÁP ---
    interval_idx = -1
    for i in range(len(x_full) - 1):
        if x_full[i] <= x_target <= x_full[i+1]:
            interval_idx = i
            break
    if interval_idx == -1: # Xử lý khi x_target nằm ngoài dải dữ liệu
        closest_abs_idx = np.argmin(np.abs(x_full - x_target))
        if closest_abs_idx == 0: interval_idx = 0
        elif closest_abs_idx == len(x_full) - 1: interval_idx = len(x_full) - 2
        else: interval_idx = closest_abs_idx if x_target > x_full[closest_abs_idx] else closest_abs_idx - 1
    
    # Stirling chọn mốc gần hơn, các PP còn lại chọn mốc trái của khoảng chứa x*
    if method_choice == '3': # Stirling
        if abs(x_target - x_full[interval_idx]) < abs(x_target - x_full[interval_idx + 1]):
            i0_abs = interval_idx
        else:
            i0_abs = interval_idx + 1
    else: # Gauss I, II, Bessel
        i0_abs = interval_idx
    
    x0 = x_full[i0_abs]
    
    # --- TRÍCH XUẤT MẢNG CON XOAY QUANH i0_abs ---
    offset = (num_points // 2) - (1 if num_points % 2 == 0 else 0)
    start_index = max(0, min(i0_abs - offset, len(x_full) - num_points))
    end_index = start_index + num_points
    x_sub, y_sub = x_full[start_index:end_index], y_full[start_index:end_index]
    
    i0 = i0_abs - start_index
    t_target = (x_target - x0) / h

    print(f"-> Khoảng chứa x* là [{x_full[interval_idx]}, {x_full[interval_idx+1]}]")
    print(f"-> Mốc gốc được chọn: x₀ = x_{i0_abs} = {x0}")
    print(f"-> Trích xuất {num_points} mốc từ index {start_index} đến {end_index - 1}")
    print(pd.DataFrame({'x': x_sub, 'y': y_sub}, index=np.arange(start_index, end_index)).to_string())
    print(f"\n-> Chỉ số gốc tương đối trong bảng trên: i0 = {i0}")
    print(f"-> Tính giá trị t_bar = (x_bar - x₀) / h = ({x_target:.8f} - {x0:.8f}) / {h:.8f} = {t_target:.8f}")

    print("\n\n=======================================================================")
    print("B2: BẢNG SAI PHÂN HỮU HẠN (Δ)")
    f_diff = np.zeros((num_points, num_points))
    f_diff[:, 0] = y_sub
    for j in range(1, num_points):
        for i in range(num_points - j):
            f_diff[i, j] = f_diff[i+1, j-1] - f_diff[i, j-1]
    df_display = pd.DataFrame(f_diff, index=x_sub)
    df_display.columns = ['y'] + [f'Δ^{i}y' for i in range(1, num_points)]
    df_display.dropna(axis=1, how='all', inplace=True)
    print(df_display.to_string())

    print("\n\n=======================================================================")
    print("B3: LẬP VECTOR D VÀ BẢNG TÍCH A CHO P(t)")
    
    is_split_method = False
    coeffs_poly_t = np.array([0.0])

    if method_choice == '3' or method_choice == '4':
        is_split_method = True
        coeffs_poly_t_even, coeffs_poly_t_odd = np.array([0.0]), np.array([0.0])

        if method_choice == '3': # STIRLING
            # --- BẬC CHẴN ---
            num_even = (num_points + 1) // 2
            coeffs_D_even, prod_table_even = np.zeros(num_even), np.zeros((num_even, num_points))
            poly_A = np.array([1.0]); max_i_even = -1
            for i in range(num_even):
                k = i * 2; idx = i0 - i
                if 0 <= idx < num_points - k:
                    coeffs_D_even[i] = f_diff[idx, k] / math.factorial(k)
                    p = poly_A; prod_table_even[i, :] = np.pad(p, (0, num_points - len(p))); max_i_even = i
                poly_A = np.polymul(poly_A, [-float(i*i), 0, 1.0])
            # --- BẬC LẺ ---
            num_odd = num_points // 2
            coeffs_D_odd, prod_table_odd = np.zeros(num_odd), np.zeros((num_odd, num_points))
            poly_A = np.array([0, 1.0]); max_i_odd = -1
            for i in range(num_odd):
                k = i * 2 + 1; idx1, idx2 = i0 - i, i0 - i - 1
                if 0 <= idx2 < num_points - k and 0 <= idx1 < num_points - k:
                    coeffs_D_odd[i] = (f_diff[idx1, k] + f_diff[idx2, k]) / (2 * math.factorial(k))
                    p = poly_A; prod_table_odd[i, :] = np.pad(p, (0, num_points - len(p))); max_i_odd = i
                poly_A = np.polymul(poly_A, [-float((i+1)**2), 0, 1.0])
        
        elif method_choice == '4': # BESSEL
            # --- BẬC CHẴN ---
            num_even = num_points // 2
            coeffs_D_even, prod_table_even = np.zeros(num_even), np.zeros((num_even, num_points))
            basis_even = [np.array([1.0])]
            max_i_even = -1
            # Tạo trước các đa thức cơ sở chẵn: 1, t(t-1), t(t-1)(t+1)(t-2), ...
            for i in range(1, num_even):
                prev_poly = basis_even[i-1]
                next_poly = np.polymul(prev_poly, [float(i-1), 1.0]) # * (t+i-1)
                next_poly = np.polymul(next_poly, [-float(i), 1.0])   # * (t-i)
                basis_even.append(next_poly)
            
            for i in range(num_even):
                k = i * 2; is_valid = False
                if i == 0:
                    if i0 + 1 < num_points:
                        coeffs_D_even[i] = (f_diff[i0, 0] + f_diff[i0+1, 0]) / 2.0; is_valid = True
                else:
                    idx1, idx2 = i0 - i + 1, i0 - i
                    if 0 <= idx2 < num_points - k and 0 <= idx1 < num_points - k:
                        coeffs_D_even[i] = (f_diff[idx1, k] + f_diff[idx2, k]) / (2 * math.factorial(k)); is_valid = True
                if is_valid:
                    p = basis_even[i]; prod_table_even[i, :] = np.pad(p, (0, num_points - len(p))); max_i_even = i

            # --- BẬC LẺ ---
            num_odd = (num_points + 1) // 2
            coeffs_D_odd, prod_table_odd = np.zeros(num_odd), np.zeros((num_odd, num_points))
            poly_t_minus_half = np.array([-0.5, 1.0]); max_i_odd = -1
            for i in range(num_odd):
                k = i * 2 + 1; idx = i0 - i
                if 0 <= idx < num_points - k:
                    coeffs_D_odd[i] = f_diff[idx, k] / math.factorial(k)
                    if i <= max_i_even:
                        # A_lẻ_i = A_chẵn_i * (t-0.5)
                        poly_A_odd = np.polymul(basis_even[i], poly_t_minus_half)
                        p = poly_A_odd; prod_table_odd[i, :] = np.pad(p, (0, num_points - len(p)))
                    max_i_odd = i

        # --- IN KẾT QUẢ TRUNG GIAN (CHUNG) ---
        if 'max_i_even' in locals() and max_i_even > -1:
            coeffs_D_even = coeffs_D_even[:max_i_even+1]; prod_table_even = prod_table_even[:max_i_even+1, :2*(max_i_even)+2]
            print("-> Vector D_chẵn:"); print(coeffs_D_even)
            print("\n-> Bảng tích A_chẵn:"); print(pd.DataFrame(np.flip(prod_table_even, axis=1)).to_string(header=False, index=False))
            coeffs_poly_t_even = coeffs_D_even @ prod_table_even
        if 'max_i_odd' in locals() and max_i_odd > -1:
            coeffs_D_odd = coeffs_D_odd[:max_i_odd+1]; prod_table_odd = prod_table_odd[:max_i_odd+1, :2*(max_i_odd)+3]
            print("\n-> Vector D_lẻ:"); print(coeffs_D_odd)
            print("\n-> Bảng tích A_lẻ:"); print(pd.DataFrame(np.flip(prod_table_odd, axis=1)).to_string(header=False, index=False))
            coeffs_poly_t_odd = coeffs_D_odd @ prod_table_odd
    
    else: # Cho Gauss
        max_k = -1; coeffs_D = np.zeros(num_points); prod_table = np.zeros((num_points, num_points)); current_poly = np.array([1.0])
        if method_choice == '1': # Gauss I
            for k in range(num_points):
                idx = i0 - (k // 2)
                if not (0 <= idx < num_points - k): continue
                coeffs_D[k] = f_diff[idx, k] / math.factorial(k)
                p = current_poly; prod_table[k, :] = np.pad(p, (0, num_points-len(p)))
                if k % 2 == 0: current_poly = np.polymul(current_poly, [-(k/2), 1])
                else: current_poly = np.polymul(current_poly, [(k+1)/2, 1])
                max_k = k
        elif method_choice == '2': # Gauss II
             for k in range(num_points):
                idx = i0 - ((k+1) // 2)
                if not (0 <= idx < num_points - k): continue
                coeffs_D[k] = f_diff[idx, k] / math.factorial(k)
                p = current_poly; prod_table[k, :] = np.pad(p, (0, num_points-len(p)))
                if k % 2 == 0: current_poly = np.polymul(current_poly, [k/2, 1])
                else: current_poly = np.polymul(current_poly, [-((k+1)/2), 1])
                max_k = k
        
        coeffs_D = coeffs_D[:max_k + 1]; prod_table = prod_table[:max_k + 1, :max_k + 2]
        print("-> Vector hệ số D (sai phân đã xử lý / k!):"); print(coeffs_D)
        print("\n-> Bảng tích A (hệ số bậc tăng dần):")
        print(pd.DataFrame(np.flip(prod_table, axis=1)).to_string(header=False, index=False))

    print("\n\n=======================================================================")
    print("B4: TÍNH HỆ SỐ P(t), ĐA THỨC VÀ KẾT LUẬN")
    print("=======================================================================")
    
    if is_split_method:
        pad_len = len(coeffs_poly_t_odd) - len(coeffs_poly_t_even)
        if pad_len > 0: coeffs_poly_t_even = np.pad(coeffs_poly_t_even, (0, pad_len))
        else: coeffs_poly_t_odd = np.pad(coeffs_poly_t_odd, (0, -pad_len))
        coeffs_poly_t = coeffs_poly_t_even + coeffs_poly_t_odd
        
        print(f"-> Đa thức P_chẵn(t):\n   {format_polynomial(coeffs_poly_t_even, 't')}")
        print(f"\n-> Đa thức P_lẻ(t):\n   {format_polynomial(coeffs_poly_t_odd, 't')}")
        print(f"\n-> Đa thức P(t) = P_chẵn(t) + P_lẻ(t):"); print(f"   P(t) = {format_polynomial(coeffs_poly_t, 't')}")
    else: 
        coeffs_poly_t = coeffs_D @ prod_table
        print("-> Tính hệ số của P(t) theo công thức P = D * A:"); print(f"   P_t = {coeffs_poly_t}")
        print("\n-> Đa thức nội suy P(t):"); print(f"   P(t) = {format_polynomial(coeffs_poly_t, 't')}")

    poly_t = P.Polynomial(coeffs_poly_t)
    poly_x = poly_t(P.Polynomial([-x0/h, 1/h]))
    standard_coeffs = poly_x.coef
    
    print("\n-> Đa thức nội suy P(x) dạng chính tắc:"); print(f"   P(x) = {format_polynomial(standard_coeffs, 'x')}")

    print("\n--- Tính toán giá trị và đạo hàm tại x_bar ---")
    while True:
        try:
            deriv_order = int(input("Nhập cấp đạo hàm n (nhập 0 để tính P(x)): ")); 
            if deriv_order >= 0: break
        except ValueError: print("Vui lòng nhập một số nguyên không âm.")

    coeffs_for_numpy = np.flip(standard_coeffs)
    result = 0.0
    if deriv_order == 0:
        result = np.polyval(coeffs_for_numpy, x_target)
    elif deriv_order < num_points:
        poly_deriv = np.polyder(coeffs_for_numpy, deriv_order)
        result = np.polyval(poly_deriv, x_target)
    
    print(f"-> P^({deriv_order})(x={x_target}) = {result:.8f}")
    
# ==============================================================================
# HÀM CHÍNH ĐIỀU PHỐI CHƯƠNG TRÌNH
# ==============================================================================

def main():
    np.set_printoptions(suppress=True, precision=8)
    pd.options.display.float_format = '{:.8f}'.format

    print("=======================================================================")
    print("CHƯƠNG TRÌNH NỘI SUY TRUNG TÂM")
    print("=======================================================================")
    file_path = "data/input_central.xlsx"

    df, msg = read_data_from_excel(file_path); print(msg)
    if df is None: sys.exit()

    x_data, y_data = df['x'].to_numpy(), df['y'].to_numpy()
    is_equally, h = check_if_equally_spaced(x_data)

    if is_equally:
        print(f"\n[PHÂN TÍCH] Dữ liệu có các mốc CÁCH ĐỀU, h ≈ {h:.6f}")
        run_central_interpolation(x_data, y_data, h)
    else:
        print("\n[LỖI] Dữ liệu có các mốc KHÔNG CÁCH ĐỀU.")
        print("--> Các phương pháp nội suy trung tâm yêu cầu mốc cách đều.")
        print("--> Vui lòng kiểm tra lại file input hoặc sử dụng Nội suy Newton/Lagrange.")
        sys.exit()

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (1284530676.py, line 132)